### "https://go.drugbank.com/structures/small_molecule_drugs/{drugbank_id}.smiles"

In [1]:
import pdb
import os
import json
import pickle
import torch
import pandas as pd

from src.model import BindingModel
from src.inference import BridgeInference
output_dir = "./data/generation_data"

In [2]:
df = pd.read_csv("./data/DrugBank/drugbank.csv")
# get protein-based therapies
protein_df = df[~df["protein_formula"].isnull()]

/tmp/ipykernel_29995/564718184.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/DrugBank/drugbank.csv")


In [3]:
protein_df

,trial_id,kind,title,description,intervention_id,drug_id,type,drugbank_id,name,state,...,withdrawn,moldb_mono_mass,moldb_inchi,moldb_inchikey,moldb_smiles,moldb_average_mass,moldb_formula,synthesis_patent_id,protein_weight_details,biotech_kind
22,NCT00001768,drug,Intravenous immunoglobulin,Intravenous immunoglobulin (IVIg) is a mixture...,fb196f4d-a20b-4837-9ff3-259998dab92e,28,BiotechDrug,DB00028,Immune Globulin Human,liquid,...,f,NaN,NaN,NaN,NaN,NaN,NaN,4082734,\N,recombinant
29,NCT00003119,biological,filgrastim,Chemotherapy-induced neutropenia (CIN) is a co...,987e85bf-c687-4d07-b99e-9d0f9418e399,99,BiotechDrug,DB00099,Filgrastim,liquid,...,f,NaN,NaN,NaN,NaN,NaN,NaN,5994518,\N,recombinant
31,NCT00002504,biological,aldesleukin,"Aldesleukin, a lymphokine, is produced by reco...",4728dea3-74c4-4a90-916f-b27dfcd7cb19,41,BiotechDrug,DB00041,Aldesleukin,liquid,...,f,NaN,NaN,NaN,NaN,NaN,NaN,4464355,\N,recombinant
60,NCT00002618,biological,filgrastim,Chemotherapy-induced neutropenia (CIN) is a co...,da39eab6-efbc-4a78-8ecb-47be70fb68c3,99,BiotechDrug,DB00099,Filgrastim,liquid,...,f,NaN,NaN,NaN,NaN,NaN,NaN,5994518,\N,recombinant
67,NCT00002674,biological,filgrastim,Chemotherapy-induced neutropenia (CIN) is a co...,af4c7bd4-6012-432a-861f-fc3894a37ada,99,BiotechDrug,DB00099,Filgrastim,liquid,...,f,NaN,NaN,NaN,NaN,NaN,NaN,5994518,\N,recombinant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201354,NCT03565666,drug,iLet Bionic Pancreas insulin-only configuratio...,Insulin aspart is a rapid-acting form of insul...,d0993239-de2f-449d-85ed-9450299f13bb,1306,BiotechDrug,DB01306,Insulin Aspart,liquid,...,f,NaN,NaN,NaN,NaN,NaN,NaN,20120214963.0,\N,recombinant
201362,NCT03555032,drug,Talimogene laherparepvec (T-VEC),Talimogene laherparepvec is an oncolytic treat...,f494d40e-9a13-44f6-866d-b6df77c439fc,13990,BiotechDrug,DB13896,Talimogene laherparepvec,NaN,...,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\N,other
201364,NCT03577782,biological,Entyvio (Vedolizumab),Vedolizumab is a recombinant humanized IgG1 mo...,21c7f3a2-5acc-4563-bab9-081d2d1e2700,9053,BiotechDrug,DB09033,Vedolizumab,solid,...,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\N,recombinant
201386,NCT03661411,drug,Alteplase,"Human tissue plasminogen activator, purified, ...",1eefe94b-c180-42b5-a87e-b467aca8f5b9,9,BiotechDrug,DB00009,Alteplase,liquid,...,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\N,recombinant


In [4]:
{
    "node_type": {
        "biological_process": 0,
        "gene/protein": 1,
        "disease": 2,
        "effect/phenotype": 3,
        "anatomy": 4,
        "molecular_function": 5,
        "drug": 6,
        "cellular_component": 7,
        "pathway": 8,
        "exposure": 9
    },
    "relation_type": {
        "expression present": 0,
        "synergistic interaction": 1,
        "interacts with": 2,
        "ppi": 3,
        "phenotype present": 4,
        "parent-child": 5,
        "associated with": 6,
        "side effect": 7,
        "contraindication": 8,
        "expression absent": 9,
        "target": 10,
        "indication": 11,
        "enzyme": 12,
        "transporter": 13,
        "off-label use": 14,
        "linked to": 15,
        "phenotype absent": 16,
        "carrier": 17
    },
    "emb_dim": {
        "molecular_function": 768,
        "gene/protein": 2560,
        "disease": 768,
        "cellular_component": 768,
        "drug": 512,
        "biological_process": 768
    }
}

{'node_type': {'biological_process': 0,
  'gene/protein': 1,
  'disease': 2,
  'effect/phenotype': 3,
  'anatomy': 4,
  'molecular_function': 5,
  'drug': 6,
  'cellular_component': 7,
  'pathway': 8,
  'exposure': 9},
 'relation_type': {'expression present': 0,
  'synergistic interaction': 1,
  'interacts with': 2,
  'ppi': 3,
  'phenotype present': 4,
  'parent-child': 5,
  'associated with': 6,
  'side effect': 7,
  'contraindication': 8,
  'expression absent': 9,
  'target': 10,
  'indication': 11,
  'enzyme': 12,
  'transporter': 13,
  'off-label use': 14,
  'linked to': 15,
  'phenotype absent': 16,
  'carrier': 17},
 'emb_dim': {'molecular_function': 768,
  'gene/protein': 2560,
  'disease': 768,
  'cellular_component': 768,
  'drug': 512,
  'biological_process': 768}}

## Protein seq input for Q&A

In [5]:
# encode it using a protein encoder
from src.protein_encoder import load_protein_model, inference
pro_model, tokenizer = load_protein_model("facebook/esm2_t36_3B_UR50D")
pro_model.to("cuda:0")
pro_model.eval()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/ec2-user/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/ec2-user/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /home/ec2-user/miniconda3/envs/py39 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at facebook/esm2_t36_3B_UR50D were not used when initializing EsmModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t36_3B_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 2560, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
    (position_embeddings): Embedding(1026, 2560, padding_idx=1)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-35): 36 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=2560, out_features=2560, bias=True)
            (key): Linear(in_features=2560, out_features=2560, bias=True)
            (value): Linear(in_features=2560, out_features=2560, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=2560, out_features=2560, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmInter

In [6]:
# load biobridge model
checkpoint_dir = "./checkpoints/bind-openke-benchmark-6-layer-unimol"
with open(os.path.join(checkpoint_dir, "model_config.json"), "r") as f:
    model_config = json.load(f)
model = BindingModel(**model_config)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, "pytorch_model.bin")))
bind_model = BridgeInference(model)

In [7]:
# load pre-encoded disease embeddings and project for retrieval
with open("./data/embeddings/esm2b_unimo_pubmedbert/disease.pkl", "rb") as f:
    dis_raw = pickle.load(f)
dis_emb = torch.tensor(dis_raw["embedding"], dtype=torch.float32)
dis_emb = bind_model.project(
    x = dis_emb,
    src_type = 2,
)
dis_raw.keys()
dis_idx = torch.tensor(dis_raw["node_index"])
# load disease node index to its descriptions
df_dis = pd.read_csv("/home/ec2-user/data/Processed/disease.csv")

/tmp/ipykernel_29995/3383629244.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  dis_emb = torch.tensor(dis_raw["embedding"], dtype=torch.float32)


In [8]:
print(dis_emb[0][:10])

tensor([-0.0100,  0.0781,  0.0178, -0.0020,  0.1084,  0.0455, -0.0712,  0.0213,
        -0.1003,  0.0073])


In [9]:
# load pre-encoded GO terms for retrieval
with open("./data/embeddings/esm2b_unimo_pubmedbert/bp.pkl", "rb") as f:
    bp_raw = pickle.load(f)
with open("./data/embeddings/esm2b_unimo_pubmedbert/mf.pkl", "rb") as f:
    mf_raw = pickle.load(f)
with open("./data/embeddings/esm2b_unimo_pubmedbert/cc.pkl", "rb") as f:
    cc_raw = pickle.load(f)
bp_emb = torch.tensor(bp_raw["embedding"], dtype=torch.float32)
mf_emb = torch.tensor(mf_raw["embedding"], dtype=torch.float32)
cc_emb = torch.tensor(cc_raw["embedding"], dtype=torch.float32)
bp_emb =  bind_model.project(x = bp_emb, src_type = 0)
mf_emb =  bind_model.project(x = mf_emb, src_type = 5)
cc_emb =  bind_model.project(x = cc_emb, src_type = 7)

bp_idx = torch.tensor(bp_raw["node_index"])
mf_idx = torch.tensor(mf_raw["node_index"])
cc_idx = torch.tensor(cc_raw["node_index"])

# load GO term descriptions
df_bp = pd.read_csv("/home/ec2-user/data/Processed/biological.csv")
df_mf = pd.read_csv("/home/ec2-user/data/Processed/molecular.csv")
df_cc = pd.read_csv("/home/ec2-user/data/Processed/cellular.csv")

In [10]:
# df_pro = pd.read_csv("/home/ec2-user/data/Processed/protein.csv")

In [1]:
# try an example protein
seq = "MPTSSSTKKTQLQLEHLLLDLQMILNGINNYKNPKLTRMLTFKFYMPKKATELKHLQCLEEELKPLEEVLNLAQSKNFHLRPRDLISNINVIVLELKGSETTFMCEYADETATIVEFLNRWITFSQSIISTLT"
name,dbid = "aldesleukin", "DB00041"

In [12]:
# not in our protein database?
# df_pro[df_pro["sequence"]  == seq]

In [13]:
def encode_protein_seq(seq):
    # encode using protein base model
    pro_model.eval()
    tokenized = tokenizer([seq], truncation=True, padding=False, return_tensors='pt')
    tokenized = {k: v.to("cuda:0") for k,v in tokenized.items()}
    emb = inference(pro_model, tokenized)
    return emb.cpu()

In [14]:
# encode raw seq into raw embeddings
raw_pro_emb = encode_protein_seq(seq)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [15]:
# transform raw seq embeddings to the disease space
tr_dis_emb = bind_model.transform(
    x = raw_pro_emb,
    src_type = 1, # protein
    tgt_type = 2, # disease
    rel_type = 6, # associated with
)
print(tr_dis_emb[0][:10])

tr_bp_emb = bind_model.transform(
    x = raw_pro_emb,
    src_type = 1, # protein
    tgt_type = 0, # bp
    rel_type = 2, # interacts with
)
print(tr_bp_emb[0][:10])


tr_mf_emb = bind_model.transform(
    x = raw_pro_emb,
    src_type = 1, # protein
    tgt_type = 5, # mf
    rel_type = 2, # interacts with
)
print(tr_mf_emb[0][:10])


tr_cc_emb = bind_model.transform(
    x = raw_pro_emb,
    src_type = 1, # protein
    tgt_type = 7, # cc
    rel_type = 2, # interacts with
)
print(tr_cc_emb[0][:10])

tensor([ 0.2283,  1.2615, -0.0261,  0.6228, -1.6979,  0.4304, -0.5485,  1.0857,
        -1.8836, -0.1505])
tensor([ 0.0244, -0.3389, -2.0661,  0.3554,  0.2586,  1.2028, -1.9445, -0.2812,
         0.9398, -0.2396])
tensor([-0.1927, -1.0443, -0.6551,  0.1085,  1.6447, -1.1099, -1.8514,  1.8590,
        -0.9917,  1.0444])
tensor([ 0.3062, -0.2222, -1.2298,  0.6733, -2.3767,  0.1999,  0.3379, -1.2253,
        -0.3016,  1.5424])


In [16]:
def retrieve_topk_disease(tgt, topk=10):
    """Args:
    tgt: emb with [1, dim]
    """
    cossim = torch.cosine_similarity(tgt, dis_emb, dim=1)
    top_k = torch.topk(cossim, k=topk, dim=0)
    retrieved_dis = df_dis.set_index("node_index").loc[dis_idx[top_k.indices].numpy()]
    # retrieved_dis = df_dis[df_dis["node_index"].isin(dis_idx[top_k.indices].numpy())]
    return retrieved_dis

def retrieve_topk_bp(tgt, topk=10):
    cossim = torch.cosine_similarity(tgt, bp_emb, dim=1)    
    top_k = torch.topk(cossim, k=topk, dim=0)
    retrieved = df_bp.set_index("node_index").loc[bp_idx[top_k.indices].numpy()]
    # retrieved = df_bp[df_bp["node_index"].isin(bp_idx[top_k.indices].numpy())]
    return retrieved

def retrieve_topk_mf(tgt, topk=10):
    cossim = torch.cosine_similarity(tgt, mf_emb, dim=1)    
    top_k = torch.topk(cossim, k=topk, dim=0)    
    # retrieved = df_mf[df_mf["node_index"].isin(mf_idx[top_k.indices].numpy())]
    retrieved = df_mf.set_index("node_index").loc[mf_idx[top_k.indices].numpy()]
    return retrieved

def retrieve_topk_cc(tgt, topk=10):
    cossim = torch.cosine_similarity(tgt, cc_emb, dim=1)
    top_k = torch.topk(cossim, k=topk, dim=0)
    # retrieved = df_cc[df_cc["node_index"].isin(cc_idx[top_k.indices].numpy())]
    retrieved = df_cc.set_index("node_index").loc[cc_idx[top_k.indices].numpy()]
    return retrieved

In [17]:
res_bp = retrieve_topk_bp(tr_bp_emb, 10)
for v in res_bp["node_name"].values:
    print(v)

positive regulation of cell population proliferation
multicellular organism development
signal transduction
spermatogenesis
cell differentiation
G protein-coupled receptor signaling pathway
negative regulation of cell population proliferation
sensory perception of sound
female pregnancy
nervous system development


In [18]:
res_mf = retrieve_topk_mf(tr_mf_emb, 10)
res_mf["node_name"].values
for v in res_mf["node_name"].values:
    print(v)

metal ion binding
protein binding
calcium ion binding
identical protein binding
protein homodimerization activity
zinc ion binding
enzyme binding
protein-containing complex binding
signaling receptor binding
protease binding


In [19]:
res_cc = retrieve_topk_cc(tr_cc_emb, 10)
res_cc["node_name"].values
for v in res_cc["node_name"].values:
    print(v)

plasma membrane
extracellular exosome
extracellular region
cytoplasm
extracellular space
apical plasma membrane
cell surface
cytosol
membrane
external side of plasma membrane


In [20]:
res_dis = retrieve_topk_disease(tr_dis_emb, 10)

In [21]:
for v in res_dis["mondo_name"].unique():
    print(v)

hepatocellular carcinoma
hepatocellular adenoma
adult hepatocellular carcinoma
hepatocellular clear cell carcinoma
liver cancer
undifferentiated carcinoma of liver and intrahepatic biliary tract
nonpapillary renal cell carcinoma
squamous cell carcinoma of liver and intrahepatic biliary tract
stroke disorder
familial prostate carcinoma
prostate cancer
adenocarcinoma of liver and intrahepatic biliary tract
drug-induced liver injury


## Galactica for generating the answer

In [2]:
import torch
from transformers import AutoTokenizer, OPTForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-120b")
# model = OPTForCausalLM.from_pretrained("facebook/galactica-120b", device_map="auto", load_in_8bit=True, cache_dir="/home/ec2-user/checkpoints")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/ec2-user/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/ec2-user/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /home/ec2-user/miniconda3/envs/py39 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


In [3]:
tokenizer = AutoTokenizer.from_pretrained("GeorgiaTechResearchInstitute/galactica-30b-evol-instruct-70k")
model = AutoModelForCausalLM.from_pretrained("GeorgiaTechResearchInstitute/galactica-30b-evol-instruct-70k", device_map="auto", torch_dtype=torch.bfloat16, cache_dir="/home/ec2-user/checkpoints")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
def generate_answer(input_text, max_new_tokens=128, temperature=0.7, top_k=40):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_new_tokens=max_new_tokens,
                            top_k=top_k,
                            do_sample=True,
                            temperature=temperature,
                            repetition_penalty=1.0,
    )

    # contrastive search
    # outputs = model.generate(
    #     input_ids,
    #     max_new_tokens=max_new_tokens,
    #     penalty_alpha=0.6, top_k=4,
    # )
    return tokenizer.decode(outputs[0])

In [5]:
prompt_template = """### Instruction:
{instruction}

### Response:"""

prompt = prompt_template.format_map(
    {"instruction":f"""Protein Sequence: [START_AMINO] {seq} [END_AMINO]

Biological process:
    positive regulation of cell population proliferation
    multicellular organism development
    signal transduction
    spermatogenesis
    cell differentiation
    G protein-coupled receptor signaling pathway
    negative regulation of cell population proliferation
    sensory perception of sound
    female pregnancy
    nervous system development

Molecular function:
    metal ion binding
    protein binding
    calcium ion binding
    identical protein binding
    protein homodimerization activity
    zinc ion binding
    enzyme binding
    protein-containing complex binding
    signaling receptor binding
    protease binding

Cellular compnent:
    plasma membrane
    extracellular exosome
    extracellular region
    cytoplasm
    extracellular space
    apical plasma membrane
    cell surface
    cytosol
    membrane
    external side of plasma membrane

Associated diseases:
    hepatocellular carcinoma
    hepatocellular adenoma
    adult hepatocellular carcinoma
    hepatocellular clear cell carcinoma
    liver cancer
    undifferentiated carcinoma of liver and intrahepatic biliary tract
    nonpapillary renal cell carcinoma
    squamous cell carcinoma of liver and intrahepatic biliary tract
    stroke disorder
    familial prostate carcinoma
    prostate cancer
    adenocarcinoma of liver and intrahepatic biliary tract
    drug-induced liver injury

Consider the associated diseases, biological process, molecular function, and cellular component, what are the main possible associated conditions and main pharmacodynamics of this protein-based therapy?""",}
)

print(prompt)


### Instruction:
Protein Sequence: [START_AMINO] MPTSSSTKKTQLQLEHLLLDLQMILNGINNYKNPKLTRMLTFKFYMPKKATELKHLQCLEEELKPLEEVLNLAQSKNFHLRPRDLISNINVIVLELKGSETTFMCEYADETATIVEFLNRWITFSQSIISTLT [END_AMINO]

Biological process:
    positive regulation of cell population proliferation
    multicellular organism development
    signal transduction
    spermatogenesis
    cell differentiation
    G protein-coupled receptor signaling pathway
    negative regulation of cell population proliferation
    sensory perception of sound
    female pregnancy
    nervous system development

Molecular function:
    metal ion binding
    protein binding
    calcium ion binding
    identical protein binding
    protein homodimerization activity
    zinc ion binding
    enzyme binding
    protein-containing complex binding
    signaling receptor binding
    protease binding

Cellular compnent:
    plasma membrane
    extracellular exosome
    extracellular region
    cytoplasm
    extracellular space
    apical pla

In [6]:
outputs = generate_answer(prompt, max_new_tokens=256, temperature=1.0, top_k=50)
print(outputs)

### Instruction:
Protein Sequence: [START_AMINO] MPTSSSTKKTQLQLEHLLLDLQMILNGINNYKNPKLTRMLTFKFYMPKKATELKHLQCLEEELKPLEEVLNLAQSKNFHLRPRDLISNINVIVLELKGSETTFMCEYADETATIVEFLNRWITFSQSIISTLT [END_AMINO]

Biological process:
    positive regulation of cell population proliferation
    multicellular organism development
    signal transduction
    spermatogenesis
    cell differentiation
    G protein-coupled receptor signaling pathway
    negative regulation of cell population proliferation
    sensory perception of sound
    female pregnancy
    nervous system development

Molecular function:
    metal ion binding
    protein binding
    calcium ion binding
    identical protein binding
    protein homodimerization activity
    zinc ion binding
    enzyme binding
    protein-containing complex binding
    signaling receptor binding
    protease binding

Cellular compnent:
    plasma membrane
    extracellular exosome
    extracellular region
    cytoplasm
    extracellular space
    apical pla

In [ ]:
hepatocellular carcinoma
hepatocellular adenoma
adult hepatocellular carcinoma
hepatocellular clear cell carcinoma
liver cancer
undifferentiated carcinoma of liver and intrahepatic biliary tract
nonpapillary renal cell carcinoma
squamous cell carcinoma of liver and intrahepatic biliary tract
stroke disorder
familial prostate carcinoma
prostate cancer
adenocarcinoma of liver and intrahepatic biliary tract
drug-induced liver injury

In [ ]:
positive regulation of cell population proliferation
multicellular organism development
signal transduction
spermatogenesis
cell differentiation
G protein-coupled receptor signaling pathway
negative regulation of cell population proliferation
sensory perception of sound
female pregnancy
nervous system development